In [45]:
import pandas as pd
import numpy as np
import re
import os
import matplotlib.pyplot as plt
import bs4
import ctypes
import requests
import time
from bs4 import BeautifulSoup
from Levenshtein import distance as levenshtein_distance

In [17]:
!pip install python-Levenshtein

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: Levenshtein
    Found existing installation: Levenshtein 0.21.1
    Uninstalling Levenshtein-0.21.1:
      Successfully uninstalled Levenshtein-0.21.1


In [11]:
#Leer el archivo con el vocabulario 
path = '/export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/models/Mallet/es_Mallet_place_without_lote_CPV_40_topics_45/model_data/TMmodel/vocab.txt'

# Lista para almacenar las palabras del archivo
palabras_archivo = []

# Leer el archivo .txt y guardar las palabras en la lista
with open(path, 'r', encoding='utf-8') as f:
    palabras_archivo = f.read().split()
    
# Mostrar las palabras del archivo
print("Palabras del archivo:", len(palabras_archivo))

Palabras del archivo: 70527


In [28]:
def agrupar_palabras(palabras):
    # Agrupar por las tres primeras letras
    agrupados_por_prefijo = {}
    # Descomponer palabras compuestas y agrupar por los prefijos de cada sub-palabra
    for palabra in palabras:
        sub_palabras = palabra.split('_')  # Divide la palabra en sub-palabras basadas en '_'
        for sub_palabra in sub_palabras:
            prefijo = sub_palabra[:3] if len(sub_palabra) >= 3 else sub_palabra
            if prefijo not in agrupados_por_prefijo:
                agrupados_por_prefijo[prefijo] = []
            agrupados_por_prefijo[prefijo].append(palabra)

    # Usamos numpy para crear un array de las palabras para iteraciones más rápidas
    palabras_array = np.array(palabras)
    n = len(palabras)
    agrupados_por_similitud = []

    # Matriz de distancia para optimizar las comparaciones
    for i in range(n):
        for j in range(i + 1, n):
            if levenshtein_distance(palabras_array[i], palabras_array[j]) <= 2:
                agrupados_por_similitud.append((palabras_array[i], palabras_array[j]))

    return agrupados_por_prefijo, agrupados_por_similitud

In [29]:
agrupados_por_prefijo, agrupados_por_similitud = agrupar_palabras(palabras_archivo[:1000])
print("Agrupados por prefijo:", agrupados_por_prefijo)

Agrupados por prefijo: {'ind': ['indebido', 'independizar', 'industrial_curiscada', 'nave_industrial', 'derivación_individual'], 'rec': ['recinto_dureta', 'recolocación', 'bomba_recirculación', 'reconstrucción', 'centro_recría', 'recaudación', 'rectorado'], 'dur': ['recinto_dureta', 'desempleo_duración', 'duración', 'duranción', 'duran_estrany', 'durar', 'duratón'], 'fil': ['filtración_cubierta', 'filtrina', 'filtración', 'evitar_filtración'], 'cub': ['filtración_cubierta', 'impermeabilización_cubierto', 'piscina_cubierto', 'peto_cubierta', 'impermeabilización_cubierta', 'cube', 'cubierta_plano', 'canalón_cubierta', 'gotera_cubierta', 'alero_cubierta'], 'par': ['parque_bomberos', 'parto', 'paramento', 'parroquia_pruvia', 'repavimentación_parcial', 'parquing', 'desbroce_parcela', 'parque_proveedores', 'pared_techo', 'iglesia_parroquial', 'parque_zoológico', 'parc_bit', 'paramento_vertical', 'parquesol', 'pareada', 'pared_cabaña', 'techo_pared'], 'bom': ['parque_bomberos', 'bomba_calor',

In [30]:
print("Agrupados por similitud:", agrupados_por_similitud)

Agrupados por similitud: [('garaje', 'pasaje'), ('silera', 'sileras'), ('duración', 'duranción'), ('duración', 'duratón'), ('mapa', 'apb'), ('mapa', 'mapama'), ('mapa', 'maz'), ('mapa', 'aph'), ('mapa', 'rampa'), ('semisotano', 'semisótano'), ('impermeabilización_cubierto', 'impermeabilización_cubierta'), ('berrón', 'butrón'), ('rellano', 'relleno'), ('rellano', 'sellado'), ('forjado', 'forrado'), ('sitar', 'optar'), ('pme', 'pcv'), ('pme', 'ave'), ('pme', 'cpm'), ('pme', 'ege'), ('parto', 'peto'), ('filtrina', 'vitrina'), ('cenca', 'cnsa'), ('cenca', 'cegc'), ('devastado', 'levantado'), ('pasillo', 'pasillos'), ('latón', 'peatón'), ('saneado', 'escaneado'), ('apb', 'atb'), ('apb', 'ave'), ('apb', 'cpm'), ('apb', 'aph'), ('apb', 'beb'), ('apb', 'apeo'), ('campùs', 'compás'), ('dgc', 'dggc'), ('dgc', 'cegc'), ('dgc', 'ege'), ('pcv', 'eco'), ('reloj', 'delo'), ('gabinete_ministro', 'gabinete_ministra'), ('mampara', 'mapama'), ('mampara', 'mámpara'), ('cuno', 'cfo'), ('cuno', 'cube'), ('e

In [31]:
agrupados_por_prefijo, agrupados_por_similitud = agrupar_palabras(palabras_archivo)
print("Agrupados por prefijo:", agrupados_por_prefijo)

Agrupados por prefijo: {'ind': ['indebido', 'independizar', 'industrial_curiscada', 'nave_industrial', 'derivación_individual', 'indicar_prescripciones', 'indirecto', 'inductor', 'independiente', 'indicación', 'inducción', 'independización', 'independencia', 'sa_indioteria', 'independiente_regulación', 'ilustrator_indesign', 'indicar', 'indiot', 'poligono_industrial', 'aplicaciones_industriales', 'industrializado', 'educativo_industrializado', 'ind_brizo', 'st_industria', 'industrias_innovación', 'ingeniero_industrial', 'laurel_india', 'consejería_industria', 'ind', 'secadora_industrial', 'lavadora_industrial', 'residencia_indistinto', 'industrial_torrehierro', 'héroes_independencia', 'individualización', 'derivació_individual', 'vestíbulo_independencia', 'indianos', 'industria_comercio', 'industrias', 'polígon_industrial', 'nau_industrial', 'inductiva', 'independizació', 'polígonos_industriales', 'indoor', 'independent', 'acometida_indorama', 'turismo_industria', 'autoridad_independie

In [33]:
print("Agrupados por similitud:", agrupados_por_similitud[0:20000])

Agrupados por similitud: [('parque_bomberos', 'parques_bomberos'), ('largas', 'claras'), ('largas', 'sardas'), ('largas', 'marzas'), ('largas', 'marges'), ('largas', 'alarmas'), ('largas', 'laxas'), ('largas', 'carpas'), ('largas', 'larna'), ('largas', 'lorga'), ('largas', 'laias'), ('largas', 'argas'), ('largas', 'mareas'), ('largas', 'augas'), ('largas', 'lañas'), ('largas', 'larga'), ('largas', 'jaras'), ('largas', 'sargal'), ('largas', 'nareas'), ('largas', 'llarga'), ('largas', 'laruns'), ('largas', 'larva'), ('largas', 'avgas'), ('largas', 'arges'), ('largas', 'sarga'), ('largas', 'jarras'), ('largas', 'martas'), ('largas', 'cargas'), ('largas', 'lagoas'), ('largas', 'zarzas'), ('largas', 'larrés'), ('largas', 'alargar'), ('largas', 'larues'), ('largas', 'arga'), ('largas', 'darías'), ('largas', 'sargos'), ('largas', 'argar'), ('largas', 'laguas'), ('largas', 'tareas'), ('cuartel', 'cuarteles'), ('cuartel', 'cantel'), ('cuartel', 'huartes'), ('cuartel', 'cuarta'), ('cuartel', 'cu

In [40]:
#Función para encontrar palabras que empiecen por san_, pº, 
def palabras_que_empiezan(lista_palabras):
    palabras_san = [palabra for palabra in lista_palabras if palabra.startswith('san_') or palabra.startswith('pº')]
    return palabras_san

palabras_san = palabras_que_empiezan(palabras_archivo)
print(palabras_san)

In [43]:
def diferir_en_una_letra(palabras):
    from collections import defaultdict

    def palabras_diferentes_en_una(palabra1, palabra2):
        """ Retorna True si las palabras difieren en exactamente una letra. """
        diferencias = 0
        for c1, c2 in zip(palabra1, palabra2):
            if c1 != c2:
                diferencias += 1
                if diferencias > 1:
                    return False
        return diferencias == 1

    # Agrupamos palabras por longitud
    palabras_por_longitud = defaultdict(list)
    for palabra in palabras:
        palabras_por_longitud[len(palabra)].append(palabra)

    pares = []

    # Comparamos solo palabras de la misma longitud
    for grupo in palabras_por_longitud.values():
        n = len(grupo)
        for i in range(n):
            for j in range(i + 1, n):
                if palabras_diferentes_en_una(grupo[i], grupo[j]):
                    pares.append((grupo[i], grupo[j]))

    return pares

resultado = diferir_en_una_letra(palabras_archivo)

In [46]:
host = 'http://dle.rae.es'
referer = 'http://www.rae.es/'
parser = 'lxml'

In [47]:
s = requests.Session()

def solveChallenge(c, slt, s1, s2, n, table):
    m = pow(ord(s2) - ord(s1) + 1, n)

    arr = [s1] * n

    for i in range(m-1):
        for j in range(n-1, -1, -1):
            arr[j] = chr(ord(arr[j]) + 1)

            if arr[j] <= s2:
                break
            else:
                arr[j] = s1

        chlg = ''.join(arr)
        stri = chlg + slt
        crc = ctypes.c_int32(0).value ^ ctypes.c_int32(-1).value

        for k in range(len(stri)):
            index = ((ctypes.c_int32(crc).value ^
                      ctypes.c_int32(ord(stri[k])).value) & 0x000000FF) * 9
            x = int(table[index:index+8], 16)
            crc = ctypes.c_int32(crc >> 8).value ^ ctypes.c_int32(x).value

        crc = abs(ctypes.c_int32(crc).value ^ ctypes.c_int32(-1).value)

        if crc == c:
            break

    return chlg


def doRequest(requestUrl, rf):
    response = s.get(requestUrl).text
    print(response)
    ## Should add a try-catch here, Response might not include the substring value.
    try:
        tmp = response.index('document.forms[0].elements[1].value=\"') + 37
        first = response[tmp:response.index(':', tmp)]
    except:
        first = ""
    
    try:
        tmp = response.index('var slt = \"') + 11
        slt = response[tmp:response.index('\"', tmp)]
    except:
        slt = ""
    
    try:
        tmp = response.index('var c = ') + 8
        c = int(response[tmp:response.index('\r', tmp)])
    except:
        c = ""
    
    try:
        tmp = response.index('var s1 = \'') + 10
        s1 = response[tmp:response.index('\'', tmp)]
    except:
        c = ""
    
    try:
        tmp = response.index('var s2 = \'') + 10
        s2 = response[tmp:response.index('\'', tmp)]
    except:
        s2 = ""
    
    try:
        tmp = response.index('var n = ') + 8
        n = int(response[tmp:response.index('\n', tmp)])
    except:
        n = ""
    
    tmp = response.index('var table = \"') + 13
    table = response[tmp:response.index('\"', tmp)]

    chlg = solveChallenge(c, slt, s1, s2, n, table)

    cr = first + ':' + chlg + ':' + slt + ':' + str(c)

    payload = (('TS017111a7_id', '3'),
               ('TS017111a7_cr', cr),
               ('TS017111a7_76', '0'),
               ('TS017111a7_86', '0'),
               ('TS017111a7_md', '1'),
               ('TS017111a7_rf', rf),
               ('TS017111a7_ct', '0'),
               ('TS017111a7_pd', '0'))

    s.headers.update({'Referer': requestUrl})
    s.headers.update({'Accept-Encoding': 'gzip, deflate'})
    s.headers.update({'Cache-Control': 'max-age=0'})
    s.headers.update({'Origin': host})

    return s.post(requestUrl, data=payload)


def conjugate(name, data, col):
    result = name + '\n'

    for i in range(0, 8):
        result += data[i][0] + ' ' + data[i][col] + '\n'

    return result


def search(word):
    s.headers.update({'Upgrade-Insecure-Requests': '1'})
    s.headers.update({'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'})
    s.headers.update({'Referer': referer})
    s.headers.update({'Accept-Language': 'es-ES,es;q=0.8'})
    s.headers.update({'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.81 Safari/537.36'})
    s.headers.update({'Accept-Encoding': 'gzip, deflate, sdch'})

    s.cookies = requests.utils.cookiejar_from_dict({'cookies_rae': 'aceptada'})

    url1 = host + '/?w=' + word
    url2 = host + '/srv/search?w=' + word
    print(url1)

    doRequest(url1, referer)          # get the cookie
    response = doRequest(url2, url2)  # get the definition
    print(response)

    soup = bs4.BeautifulSoup(response.text, parser)

    f0 = soup.find('div', id='f0')

    if f0:  # this exists if the word was not found
        result = f0.find('span').text
    else:
        article = soup.find('article')

        if article:
            result = article.get_text()
        else:
            options = soup.findAll('a')
            links = []
            i = 1

            for op in options:
                links.append(op.get('href'))
                print(str(i) + '. ' + op.text)
                i += 1

            ## This should not depend on user interaction.
            opt = int(input('\nSeleccione una opcion: ')) - 1

            url3 = host + '/srv/' + links[opt]

            response = s.get(url3)
            soup = bs4.BeautifulSoup(response.text, parser)
            result = soup.find('article').get_text()

        e2 = soup.find('a', class_='e2')

        if e2:
            url4 = host + '/srv/' + e2['href']

            time.sleep(3)

            response = s.get(url4)
            soup = bs4.BeautifulSoup(response.text, parser)

            rows = soup.find('table', class_='cnj').find_all('tr')

            data = []
            headers = []

            for row in rows:
                cells = [cell.text.strip() for cell in row.find_all('td')]
                data.append([cell for cell in cells if cell])
                heads = [header.text.strip() for header in row.find_all('th')]
                headers.append([header for header in heads if header])

            data = [e for e in data if e]

            conjugation = headers[0][0] + '\n'
            conjugation += headers[1][0] + ': ' + data[0][0] + '\n'
            conjugation += headers[1][1] + ': ' + data[0][1] + '\n'
            conjugation += headers[3][0] + ': ' + data[1][0] + '\n'
            conjugation += '\n'
            conjugation += conjugate(headers[5][0] + ' ' + headers[6][3], data[2:10], 1) + '\n'
            conjugation += conjugate(headers[5][0] + ' ' + headers[6][4], data[2:10], 2) + '\n'
            conjugation += conjugate(headers[5][0] + ' ' + headers[15][0], data[10:18], 1) + '\n'
            conjugation += conjugate(headers[5][0] + ' ' + headers[15][1], data[10:18], 2) + '\n'
            conjugation += conjugate(headers[5][0] + ' ' + headers[24][0], data[18:26], 1) + '\n'
            conjugation += conjugate(headers[33][0] + ' ' + headers[34][3], data[26: 34], 1) + '\n'
            conjugation += conjugate(headers[33][0] + ' ' + headers[34][4], data[26: 34], 2) + '\n'
            conjugation += conjugate(headers[33][0] + ' ' + headers[43][0], data[34:42], 1) + '\n'
            conjugation += headers[52][0] + '\n'
            conjugation += data[42][0] + ' ' + data[42][1] + '\n'
            conjugation += data[43][0] + ' ' + data[43][1] + '\n'
            conjugation += data[44][0] + ' ' + data[44][1] + '\n'
            conjugation += data[45][0] + ' ' + data[45][1]

            result += '\n' + conjugation + '\n'

    return result

In [92]:
def search(word):
    s.headers.update({'Upgrade-Insecure-Requests': '1'})
    s.headers.update({'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'})
    s.headers.update({'Referer': referer})
    s.headers.update({'Accept-Language': 'es-ES,es;q=0.8'})
    s.headers.update({'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.81 Safari/537.36'})
    s.headers.update({'Accept-Encoding': 'gzip, deflate, sdch'})

    s.cookies = requests.utils.cookiejar_from_dict({'cookies_rae': 'aceptada'})

    url1 = host + '/?w=' + word
    url2 = host + '/srv/search?w=' + word
    #print(url1)

    response = s.get(url1).text
    soup = BeautifulSoup(response, parser)
    text = soup.get_text()
    is_in_dict = f"La palabra {word} no está en el Diccionario"
    if is_in_dict in text:
        return False
    else:
        return True

In [64]:
resultado

[('ceramica', 'cerámica'),
 ('atalayas', 'atalayes'),
 ('duración', 'duracion'),
 ('luminoso', 'luminosa'),
 ('corquera', 'porquera'),
 ('corquera', 'horquera'),
 ('secadero', 'recadero'),
 ('semáforo', 'semaforo'),
 ('glorieta', 'gloriete'),
 ('ciervana', 'ciérvana'),
 ('cabecera', 'cabecers'),
 ('cabecera', 'cabecero'),
 ('parquing', 'pàrquing'),
 ('parquing', 'párquing'),
 ('manguera', 'mangueta'),
 ('términco', 'términio'),
 ('rodadura', 'rozadura'),
 ('pantalan', 'pantalón'),
 ('imbornal', 'inbornal'),
 ('reparado', 'separado'),
 ('reparado', 'reparade'),
 ('reparado', 'reparada'),
 ('ventanal', 'ventanas'),
 ('diamants', 'diamante'),
 ('piscinas', 'pîscinas'),
 ('piscinas', 'piscines'),
 ('campanas', 'campanar'),
 ('campanas', 'campañas'),
 ('campanas', 'campanal'),
 ('melamina', 'melanina'),
 ('colombia', 'columbia'),
 ('fregador', 'fresador'),
 ('anomalía', 'anomalia'),
 ('efectuar', 'efectúar'),
 ('invierno', 'infierno'),
 ('palmeras', 'palmeral'),
 ('palmeras', 'palmeres'),
 

In [86]:
len(resultado)

19871

In [93]:
# Lista para almacenar las filas del DataFrame
data = []
# Lista para almacenar posibles stopwords
posibles_stopwords = []

# Procesar cada tupla en resultado
for palabra1, palabra2 in resultado:
    #print("La palabra1 es:\n", palabra1)
    #print("La palabra2 es:\n", palabra2)
    existe_1 = search(palabra1)
    #print("La palabra1 existe?", existe_1)
    existe_2 = search(palabra2)
    #print("La palabra2 existe?", existe_2)
  
    # Añadir al DataFrame si una existe y la otra no
    if (existe_1 and not existe_2) or (not existe_1 and existe_2):
        equivalencia = f"{palabra1}:{palabra2}" if not existe_1 else f"{palabra2}:{palabra1}"
        data.append({
            'columna1': palabra1,
            'columna2': palabra2,
            'existe_palabra1': existe_1,
            'existe_palabra2': existe_2,
            'equivalencias': equivalencia
        })
    # Si ninguna de las dos palabras existe, las añadimos a posibles_stopwords
    elif not existe_1 and not existe_2:
        posibles_stopwords.append(palabra1)
        posibles_stopwords.append(palabra2)

# Crear DataFrame
df = pd.DataFrame(data, columns=['columna1', 'columna2', 'existe_palabra1', 'existe_palabra2', 'equivalencias'])
# Añadir la columna de posibles_stopwords
df['posibles_stopwords'] = ', '.join(posibles_stopwords)

In [94]:
df

,columna1,columna2,existe_palabra1,existe_palabra2,equivalencias,posibles_stopwords
0,ceramica,cerámica,False,True,ceramica:cerámica,"corquera, horquera, ciervana, ciérvana, parqui..."
1,duración,duracion,True,False,duracion:duración,"corquera, horquera, ciervana, ciérvana, parqui..."
2,corquera,porquera,False,True,corquera:porquera,"corquera, horquera, ciervana, ciérvana, parqui..."
3,semáforo,semaforo,True,False,semaforo:semáforo,"corquera, horquera, ciervana, ciérvana, parqui..."
4,glorieta,gloriete,True,False,gloriete:glorieta,"corquera, horquera, ciervana, ciérvana, parqui..."
...,...,...,...,...,...,...
5403,hum,hun,True,False,hun:hum,"corquera, horquera, ciervana, ciérvana, parqui..."
5404,pet,pef,True,False,pef:pet,"corquera, horquera, ciervana, ciérvana, parqui..."
5405,pet,plt,True,False,plt:pet,"corquera, horquera, ciervana, ciérvana, parqui..."
5406,gau,gap,False,True,gau:gap,"corquera, horquera, ciervana, ciérvana, parqui..."


In [97]:
lista_equivalencias = df['equivalencias'].tolist()

In [98]:
lista_equivalencias

['ceramica:cerámica',
 'duracion:duración',
 'corquera:porquera',
 'semaforo:semáforo',
 'gloriete:glorieta',
 'cabecers:cabecera',
 'pantalan:pantalón',
 'inbornal:imbornal',
 'reparade:reparado',
 'diamants:diamante',
 'pîscinas:piscinas',
 'piscines:piscinas',
 'campanar:campanas',
 'campanal:campanas',
 'anomalia:anomalía',
 'efectúar:efectuar',
 'palmeres:palmeras',
 'palmerar:palmeras',
 'templeto:templete',
 'asfaltat:asfaltar',
 'barriado:barriada',
 'separato:separado',
 'tecnicas:técnicas',
 'planicia:planicie',
 'acustica:acústica',
 'restande:restante',
 'acustico:acústico',
 'acúatico:acústico',
 'acustico:acuático',
 'pirineus:pirineos',
 'ludoteka:ludoteca',
 'fiscalia:fiscalía',
 'cláustro:claustro',
 'maritimo:marítimo',
 'remocion:remoción',
 'division:división',
 'sotonera:botonera',
 'ensolado:ensilado',
 'gimnasió:gimnasia',
 'fijacion:fijación',
 'satelite:satélite',
 'palmerar:palmeral',
 'porteria:portería',
 'chorrero:chorrera',
 'avenidad:avenidas',
 'jardiner

In [99]:
# Especifica la ruta donde quieres guardar el archivo de texto
ruta_archivo_txt = '/export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/rae_lista_equivalencias.txt'

# Abre el archivo de texto en modo escritura
with open(ruta_archivo_txt, 'w', encoding='utf-8') as archivo_txt:
    # Itera sobre cada elemento de la lista
    for palabra in lista_equivalencias:
        # Escribe la palabra en el archivo de texto con un salto de línea
        #archivo_txt.write(palabra + '\n')

In [96]:
# Especifica la ruta donde quieres guardar el archivo Parquet
ruta = '/export/usuarios_ml4ds/cggamella/NP-Search-Tool/sample_data/posible_lista_equivalencias.parquet'

# Guarda el DataFrame como un archivo Parquet en la ruta especificada
#df.to_parquet(ruta)